## 3.7 Fixpunktverfahren zum Lösen linearer Gleichungssysteme

In [ ]:
import numpy as np

**Die einfache Richardson-Iteration**

In [ ]:
def richardson(A, b, x, it=1000, omega=1, tol=1e-5):
    x = x.copy()
    for i in range(it):
        w = b - np.dot(A, x)
        if np.linalg.norm(w) < tol:
            break
        x += omega * w
    return x, i

Wir testen die einfache Richardson-Iteration anhand des linearen Gleichungssystems $Ax=b$ mit
$$A=\begin{pmatrix}3 & 1.8 & 1\\ 1.4 & 2.3 & -0.7\\ 0.8 & 0.3 & 1.5 \end{pmatrix}\qquad
b = \begin{pmatrix} 1.2\\-2.1\\0.6\end{pmatrix}.$$
Mit `numpy` haben wir dabaei folgende 'exakte' Lösung:

In [ ]:
A = np.array([[3.0, 1.8, 1],
              [1.4, 2.3, -0.7],
              [0.8, 0.3, 1.5]])
b = np.array([1.2, -2.1, 0.6])

x_np = np.linalg.solve(A, b)
print(x_np)

Wir wenden nun die Richardson-Iteration auf dieses System mit Relaxationsparameter $\omega=1$ an und nehmen den Startvektor $x_0 = (1, -1, 0)$, der sogar recht nahe an der Lösung ist.

In [ ]:
x0 = np.array([1.0, -1.0, 0.0])

In [ ]:
x, n = richardson(A, b, x0, it=100, omega=1)
print(f'x = {x} nach {n} Schritten')
print(f'||x - x_ex||_2 = {np.linalg.norm(x - x_np)}')

Leider scheint das Verfahren zu divergieren. Nehmen wir aber einen kleineren Relaxationsparameter, können wir Konvergenz erhalten:

In [ ]:
x, n = richardson(A, b, x0, it=50, omega=0.4)
print(f'x = {x} nach {n} Schritten')
print(f'||x - x_ex||_2 = {np.linalg.norm(x - x_np)}')

**Das Jacobi-Verfahren**

Diese Implementierung ist für `numpy` **nicht optimal**, da wir z.B. die Matrix-Vektor-Produkte selber berechnen und nicht die hierfür von `numpy` zur Verfügung gestellten Routinen verwenden. Wir nutzen diese Implementierungen um die Unterschiede zwischen dem Jacobi- und Gauß-Seidel-Verfahren zu verdeutlichen.

In [ ]:
def jacobi(A, b, x, it=1000, tol=1e-5):
    n, m = A.shape
    x, x_neu = x.copy(), x.copy()
    for k in range(it):
        if np.linalg.norm(b - np.dot(A, x)) < tol:
            break
        for i in range(n):
            s = 0
            for j in range(i):
                s += A[i, j] * x[j]
            for j in range(i + 1, n):
                s += A[i, j] * x[j]
            x_neu[i] = (b[i] - s) / A[i, i]
        x[:] = x_neu
    return x, k

Angewandt auf unser obiges Beispiel erhalten wir dann

In [ ]:
x, n = jacobi(A, b, x0, it=50)
print(f'x = {x} nach {n} Schritten')
print(f'||x - x_ex||_2 = {np.linalg.norm(x - x_np)}')

**Das Gauß-Seidel Verfahren**

In [ ]:
def gauss_seidel(A, b, x, it=100, tol=1e-5):
    n, m = A.shape
    x, x_neu = x.copy(), x.copy()
    for k in range(it):
        if np.linalg.norm(b - np.dot(A, x)) < tol:
            break
        for i in range(n):
            s = 0
            for j in range(i):
                s += A[i, j] * x_neu[j]
            for j in range(i + 1, n):
                s += A[i, j] * x[j]
            x_neu[i] = (b[i] - s) / A[i, i]
        x[:] = x_neu
    return x, k

Angewandt auf unser obiges Beispiel erhalten wir dann

In [ ]:
x, n = gauss_seidel(A, b, x0, it=50)
print(f'x = {x} nach {n} Schritten')
print(f'||x - x_ex||_2 = {np.linalg.norm(x - x_np)}')

Das Gauß-Seidel-Verfahren benötigt also in etwa nur die Hälfte der Schritte wie das Jacobi-Verfahren.

### 3.7.1 Konvergenzkriterium für Jacobi- und Gauß-Seidel-Iteration

#### Beispiel 8.12 (Jacobi- und Gauß-Seidel-Verfahren bei der Modellmatrix)

Wir betrachten das lineare Gleichungssystem $Ax=b$ mit der Modellmatrix $A\in\mathbb{R}^{n\times n}$ 
$$ A = \begin{pmatrix}2 & -1 \\ -1 & 2 & -1 \\ & \ddots & \ddots & \ddots \\ && -1 & 2 & -1\\ &&& -1 & 2 \end{pmatrix}$$
sowie der rechten Seite $b\in\mathbb{R}^n$ mit $b=(1,\dots,1)^T$. Die Matrix ist also irreduzibel, des Weiteren diagonaldominant und in erster und letzter Zeile auch stark diagonaldominant. Die Jacobi- und Gauß-Seidel-Verfahren konvergieren.


In [ ]:
import time

In [ ]:
for i in range(4):
    n = 10 * 2**i
    A = np.diag(2 * np.ones(n), k=0) + np.diag(-1 * np.ones(n - 1), k=1) + np.diag(-1 * np.ones(n - 1), k=-1)
    b = np.ones(n)
    x0 = np.zeros(n)
    
    t = time.perf_counter()
    x, m = jacobi(A, b, x0, it=int(2e6), tol=1e-4)
    t = time.perf_counter() - t
    
    res = np.linalg.norm(b - np.dot(A, x))
    print(f'Jacobi: n = {n:03d}, Schritte = {m:07d} time = {t:07.3f}sec, res = {res:4.2e}')

In [ ]:
for i in range(4):
    n = 10 * 2**i
    A = np.diag(2 * np.ones(n), k=0) + np.diag(-1 * np.ones(n - 1), k=1) + np.diag(-1 * np.ones(n - 1), k=-1)
    b = np.ones(n)
    x0 = np.zeros(n)
    
    t = time.perf_counter()
    x, m = gauss_seidel(A, b, x0, it=int(2e6), tol=1e-4)
    t = time.perf_counter() - t
    
    res = np.linalg.norm(b - np.dot(A, x))
    print(f'Gauß-Seidel: n = {n:03d}, Schritte = {m:07d} time = {t:07.3f}sec, res = {res:4.2e}')

Hier sehen wir, dass das Gauß-Seidel-Verfahren fast genau halb so viele Schritte wie das Jacobi-Verfahren benötigt und somit bei gleich effizienter Implementierung doppelt so schnell ist.

Wir können beide Verfahren mit `numpy` auch effizienter implementieren, indem wir die vorhandenen Routinen für Skalarprodukte verwenden.

In [ ]:
def jacobi_np(A, b, x, it=1000, tol=1e-5):
    n, m = A.shape
    d = np.diag(A)
    x = x.copy()
    for k in range(it):
        res = b - A.dot(x)
        if np.linalg.norm(res) < tol:
            break
        x += res / d
    return x, k

In [ ]:
def gauss_seidel_np(A, b, x, it=100, tol=1e-5):
    n, m = A.shape
    x = x.copy()
    for k in range(it):
        if np.linalg.norm(b - np.dot(A, x)) < tol:
            break
        x_alt = x.copy()
        for i in range(n):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x_alt[i + 1:])
            x[i] = (b[i] - s1 - s2) / A[i, i]
    return x, k

In [ ]:
for i in range(6):
    n = 10 * 2**i
    A = np.diag(2 * np.ones(n), k=0) + np.diag(-1 * np.ones(n - 1), k=1) + np.diag(-1 * np.ones(n - 1), k=-1)
    b = np.ones(n)
    x0 = np.zeros(n)
    
    t = time.perf_counter()
    x, m = jacobi_np(A, b, x0, it=int(2e6), tol=1e-4)
    t = time.perf_counter() - t
    
    res = np.linalg.norm(b - np.dot(A, x))
    print(f'Jacobi: n = {n:03d}, Schritte = {m:07d} time = {t:07.3f}sec, res = {res:4.2e}')

In [ ]:
for i in range(5):
    n = 10 * 2**i
    A = np.diag(2 * np.ones(n), k=0) + np.diag(-1 * np.ones(n - 1), k=1) + np.diag(-1 * np.ones(n - 1), k=-1)
    b = np.ones(n)
    x0 = np.zeros(n)
    
    t = time.perf_counter()
    x, m = gauss_seidel_np(A, b, x0, it=int(1e6), tol=1e-4)
    t = time.perf_counter() - t
    
    res = np.linalg.norm(b - np.dot(A, x))
    print(f'Gauß-Seidel: n = {n:03d}, Schritte = {m:07d} time = {t:07.3f}sec, res = {res:4.2e}')

Wir sehen also, dass der Aufwand auf die einzelnen Einträge von Python aus zuzugreifen, und diese zu verändern, deutlich teurer ist, als die interne Berechnung des Matrix-Vektor-Produktes. Damit ist jetzt das Jacobi-Verfahren schneller, obwohl doppelt so viele Schritte verwendet werden. In dem wir auf die Matrix-Form des Verfahrens zurückgreifen, und das Vorwärtseinsetzen aus `scipy` verwenden, können wir mit dem Gauß-Seidel eine vergleichbare Effizienz erreichen.

In [ ]:
import scipy as sp

def gauss_seidel_sp(A, b, x, it=100, tol=1e-5):
    n, m = A.shape
    x = x.copy()
    R = np.triu(A, 1)
    LD = np.tril(A, 0)
    for k in range(it):
        if np.linalg.norm(b - np.dot(A, x)) < tol:
            break
        x = sp.linalg.solve_triangular(LD, b - np.dot(R, x), lower=True)
    return x, k

In [ ]:
for i in range(5):
    n = 10 * 2**i
    A = np.diag(2 * np.ones(n), k=0) + np.diag(-1 * np.ones(n - 1), k=1) + np.diag(-1 * np.ones(n - 1), k=-1)
    b = np.ones(n)
    x0 = np.zeros(n)
    
    t = time.perf_counter()
    x, m = gauss_seidel_sp(A, b, x0, it=int(1e6), tol=1e-4)
    t = time.perf_counter() - t
    
    res = np.linalg.norm(b - np.dot(A, x))
    print(f'Gauß-Seidel: n = {n:03d}, Schritte = {m:07d} time = {t:07.3f}sec, res = {res:4.2e}')

Hier durch wird noch einmal deutlich, dass die effiziente Umsetzung eines Algorithmus eine zentrale Rolle spielt.

### 3.7.2 Relaxationsverfahren: Das SOR-Verfahren

Wir implementieren das SOR-Verfahren in der Index-Form, damit die Effizienz direkt mit den Index-Implementierungen der Jacobi- und Gauß-Seidel-Verfahren verglichen werden kann.

In [ ]:
def sor(A, b, x, omega, it=100, tol=1e-5):
    assert (omega > 0 and omega < 2), 'Omega nicht im Intervall (0, 2)'
    n, m = A.shape
    x, x_neu = x.copy(), x.copy()
    for k in range(it):
        if np.linalg.norm(b - np.dot(A, x)) < tol:
            break
        for i in range(n):
            s = 0
            for j in range(i):
                s += A[i, j] * x_neu[j]
            for j in range(i + 1, n):
                s += A[i, j] * x[j]
            x_neu[i] = omega * (b[i] - s) / A[i, i] + (1 - omega) * x[i]
        x[:] = x_neu
    return x, k

Dazu betrachten wir zunächst wieder das lineare Gleichungssystem $Ax=b$ mit
$$A=\begin{pmatrix}3 & 1.8 & 1\\ 1.4 & 2.3 & -0.7\\ 0.8 & 0.3 & 1.5 \end{pmatrix}\qquad
b = \begin{pmatrix} 1.2\\-2.1\\0.6\end{pmatrix}.$$

In [ ]:
A = np.array([[3.0, 1.8, 1],
              [1.4, 2.3, -0.7],
              [0.8, 0.3, 1.5]])
b = np.array([1.2, -2.1, 0.6])

x_np = np.linalg.solve(A, b)

x0 = np.array([1.0, -1.0, 0.0])
x, n = sor(A, b, x0, it=100, omega=1.2)
print(f'x = {x} nach {n} Schritten')
print(f'||x - x_ex||_2 = {np.linalg.norm(x - x_np)}')

Das Verfahren konvergiert also schneller als die bisherigen Verfahren. Dies hängt aber wesentlich von der korrekten Wahl von $\omega$ ab. Probieren Sie bitte einmal andere Werte für `omega`.

#### Beispiel 8.14 (Modellmatrix mit SOR-Verfahren)

Wir kehren wieder zur Modellmatrix zurück

In [ ]:
for i in range(5):
    n = 10 * 2**i
    A = np.diag(2 * np.ones(n), k=0) + np.diag(-1 * np.ones(n - 1), k=1) + np.diag(-1 * np.ones(n - 1), k=-1)
    b = np.ones(n)
    x0 = np.zeros(n)
    
    lam = 1 - np.pi**2 / (2 * (n+1)**2)
    omega = 2 * (1 - np.sqrt(1 - lam**2)) / lam**2
    
    t = time.perf_counter()
    x, m = sor(A, b, x0, omega=omega, it=int(2e6), tol=1e-4)
    t = time.perf_counter() - t
    
    res = np.linalg.norm(b - np.dot(A, x))
    print(f'SOR: n = {n:03d}, Schritte = {m:07d} time = {t:07.3f}sec, res = {res:4.2e}')

Bei $n=80$ ist das SOR also in etwa 40 mal schneller als das Gauß-Seidl-Verfahren.

Wir können die Implementierung wieder mit `numpy` etwas verbessern 

In [ ]:
def sor_np(A, b, x, omega, it=100, tol=1e-5):
    n, m = A.shape
    x, x_alt = x.copy(), x.copy()
    for k in range(it):
        if np.linalg.norm(b - np.dot(A, x)) < tol:
            break
        x_alt[:] = x
        for i in range(n):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x_alt[i + 1:])
            x[i] = omega * (b[i] - s1 - s2) / A[i, i] + (1 - omega) * x_alt[i]
    return x, k

In [ ]:
for i in range(6):
    n = 10 * 2**i
    A = np.diag(2 * np.ones(n), k=0) + np.diag(-1 * np.ones(n - 1), k=1) + np.diag(-1 * np.ones(n - 1), k=-1)
    b = np.ones(n)
    x0 = np.zeros(n)
    
    lam = 1 - np.pi**2 / (2 * (n+1)**2)
    omega = 2 * (1 - np.sqrt(1 - lam**2)) / lam**2
    
    t = time.perf_counter()
    x, m = sor_np(A, b, x0, omega=omega, it=int(2e6), tol=1e-4)
    t = time.perf_counter() - t
    
    res = np.linalg.norm(b - np.dot(A, x))
    print(f'SOR: n = {n:03d}, Schritte = {m:07d} time = {t:07.3f}sec, res = {res:4.2e}')

Durch die erhebliche Reduktion der Anzahl der notwendigen Schritte, ist das SOR-Verfahren sogar schneller als das Jacobi-Verfahren mit `numpy` Matrix-Vektor-Produkten, obwohl wir hier wieder auf einzelne Einträge zugreifen müssen. 